In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
kwargs = {'hard_time': 0.33333333333333326, 'hard_gamma_inner': -1.5, 'hard_gamma_outer': 1.0, 'hard_rchar': 1.5, 'gsmf_phi0': -2.7800000000000002, 'mmb_amp': 390000000.0}

SAM_SHAPE = 50

DEF_NUM_REALS = 100
DEF_NUM_FBINS = 40
DEF_PTA_DUR = 16.03     # [yrs]

time, gamma_inner, gamma_outer, rchar, gsmf_phi0, mmb_amp = kwargs.values()
time = (10.0 ** time) * GYR
rchar = (10.0 ** rchar) * PC
mmb_amp = mmb_amp*MSOL

gsmf = holo.sam.GSMF_Schechter(phi0=gsmf_phi0)
gpf = holo.sam.GPF_Power_Law()
gmt = holo.sam.GMT_Power_Law()
mmbulge = holo.relations.MMBulge_KH2013(mamp=mmb_amp)

sam = holo.sam.Semi_Analytic_Model(
    gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge,
    shape=SAM_SHAPE
)
hard = holo.evolution.Fixed_Time.from_sam(
    sam, time, rchar=rchar, gamma_sc=gamma_inner, gamma_df=gamma_outer,
    exact=True, progress=False
)

In [ ]:
np.random.poisson(np.nan)

In [ ]:
pta_dur = DEF_PTA_DUR * YR
nfreqs = DEF_NUM_FBINS
hifr = nfreqs/pta_dur
pta_cad = 1.0 / (2 * hifr)
fobs_cents = holo.utils.nyquist_freqs(pta_dur, pta_cad)
fobs_edges = holo.utils.nyquist_freqs_edges(pta_dur, pta_cad)

In [ ]:
np.isnan(hard._norm)

In [ ]:
print(f"{time/GYR=:.4e} {rchar/PC=:.4e}, {gamma_inner=:.4f}, {gamma_outer=:.4f}")
test = holo.evolution.Fixed_Time([time], [1.893e43], [0.3924], [0.01], [1e4*PC], rchar, gamma_inner, 1.0, False, True)
# np.isnan(hard.dadt(1.893e43, 0.3924, 3.17857e17)).shape

In [ ]:
# (M*Q*Z,) 1D arrays of each total-mass, mass-ratio, and redshift
mt, mr, rz = [gg.ravel() for gg in sam.grid]
# ---- Get the differential-number of binaries for each bin
# convert to orbital-frequency (from GW-frequency)
# fobs_orb_edges = fobs_edges / 2.0
fobs_orb_cents = fobs_cents / 2.0

frst_orb = fobs_orb_cents[:, np.newaxis] * (1.0 + rz[np.newaxis, :])
sa = utils.kepler_sepa_from_freq(mt[np.newaxis, :], frst_orb)

# (X, M*Q*Z), hardening rate, negative values, units of [cm/sec]
mt, mr, sa, rz = np.broadcast_arrays(mt[np.newaxis, :], mr[np.newaxis, :], sa, rz[np.newaxis, :])
dadt = hard.dadt(mt, mr, sa)

bads = np.isnan(dadt)
if np.any(bads):
    print(f"BADS: {utils.frac_str(bads)}")
    print(f"{utils.minmax(mt)=}")
    print(f"{utils.minmax(mr)=}")
    print(f"{utils.minmax(sa)=}")
    print(f"{utils.stats(mt[bads])=}")
    print(f"{utils.stats(mr[bads])=}")
    print(f"{utils.stats(sa[bads])=}")
    print(f"{mt[bads][:5]=}")
    print(f"{mr[bads][:5]=}")
    print(f"{sa[bads][:5]=}")
    print(f"{rz[bads][:5]=}")

    raise

In [ ]:
gwb = sam.gwb(fobs_edges, realize=DEF_NUM_REALS, hard=hard)

In [ ]:
# fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/cori_sam-lib_2022-12-04_01.hdf5"
# fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/2022-12-05_01/sam_lib.hdf5"
fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/2022-12-05_01/sam_lib.hdf5"
data = h5py.File(fname, 'r')
print("Keys:", data.keys())
for kk, vv in data.items():
    print("\t", kk, vv.shape)
print("Attributes:")
for kk, vv in data.attrs.items():
    print("\t", kk, vv)

In [ ]:
import zcode.math as zmath

In [ ]:
fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax()


yy = data['gwb'][:]
# yy = np.median(yy, axis=-1)
ax.plot(xx, np.moveaxis(yy, 1, 0).reshape(yy.shape[1], -1), alpha=0.1, color='k', lw=0.5)

# ff = zmath.argnearest(xx, 1.0)
# yy = data['gwb'][:, ff, :].flatten()
# yy = np.log10(yy)
# kale.dist1d(yy, ax=ax, density=True, carpet=True)

plt.show()

In [ ]:
param_names = data.attrs['param_names']
for ii, par in enumerate(param_names):
    vals = data['params'][:, ii]
    print(f"{ii} {par} : [{vals.min():.2e}, {vals.max():.2e}] {utils.stats(vals)}")

In [ ]:
gwb.max()

In [ ]:
gwb = data['gwb'][:, 0, :].flatten()
print(utils.stats(gwb))
# kale.dist1d(np.log10(gwb))

In [ ]:
kale.corner(data['params'][:].T)